In [2]:
!pip install torch torchvision


     |████████████████████████████████| 68.6 MB 6.9 MB/s eta 0:00:011
     |████████████████████████████████| 1.9 MB 12.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.3 MB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 194 kB 15.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 13.7 MB/s eta 0:00:01
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl (5.3 MB)
     |████████████████████████████████| 536 kB 11.5 MB/s eta 0:00:01
You should consider upgrading via the '/Users/paidamoyo/Documents/WRN-CIFAR10-Reproduction-/venv/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


## Defining the Basic Block (Residual Block)

In [2]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, drop_rate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.drop_rate = drop_rate
        self.equalInOut = (in_planes == out_planes)
        self.shortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes,
                                                            kernel_size=1, stride=stride,
                                                            padding=0, bias=False) or None

    def forward(self, x):
        if not self.equalInOut:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.drop_rate > 0:
            out = F.dropout(out, p=self.drop_rate, training=self.training)
        out = self.conv2(out)
        return out + (x if self.equalInOut else self.shortcut(x))


## Defining Network Block (a group of residual blocks)

In [3]:
class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, drop_rate):
        super(NetworkBlock, self).__init__()
        layers = []
        for i in range(nb_layers):
            layers.append(block(i == 0 and in_planes or out_planes,
                                out_planes,
                                i == 0 and stride or 1,
                                drop_rate))
        self.layer = nn.Sequential(*layers)

    def forward(self, x):
        return self.layer(x)


## Defining the WideResNet

In [4]:
class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor=10, num_classes=10, drop_rate=0.0):
        super(WideResNet, self).__init__()
        nChannels = [16, 16*widen_factor, 32*widen_factor, 64*widen_factor]
        assert ((depth - 4) % 6 == 0)
        n = (depth - 4) // 6

        block = BasicBlock
        self.conv1 = nn.Conv2d(3, nChannels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.block1 = NetworkBlock(n, nChannels[0], nChannels[1], block, 1, drop_rate)
        self.block2 = NetworkBlock(n, nChannels[1], nChannels[2], block, 2, drop_rate)
        self.block3 = NetworkBlock(n, nChannels[2], nChannels[3], block, 2, drop_rate)
        self.bn1 = nn.BatchNorm2d(nChannels[3])
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(nChannels[3], num_classes)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, out.size()[2:])  # Global average pooling
        out = out.view(out.size(0), -1)
        return self.fc(out)


## Test the Model with Dummy Data

In [5]:
# Instantiating the model
model = WideResNet(depth=28, widen_factor=10, num_classes=10)
print("✅ WRN-28-10 model created!")

#  dummy input Test
x = torch.randn(1, 3, 32, 32)  # CIFAR-10 image size
y = model(x)
print(f"Output shape: {y.shape}")  # Should be [1, 10]


✅ WRN-28-10 model created!
Output shape: torch.Size([1, 10])
